# 杠杆交易演示

>“杠杆资产”，杠杆倍数、爆仓说明、做空操作、利息计算及还币规则等

In [1]:
from Leverage import *

以 BTC/USDT 交易对为例

In [2]:
leverage_demo = Leverage('btc', 'usdt')

总资产 **total_asset** ：杠杆账户内资产的总和，包括可用资产+冻结资产，冻结资产一般指未成交单中的资产

In [3]:
print(leverage_demo.total_asset_1, 'BTC', ',', leverage_demo.total_asset_2, 'USDT')

0 BTC , 0 USDT


转入资产 **deposit** ：从其他账户通过资金划转转入杠杆账户里的币

> 每个交易队构建一个单独的杠杆账户，只有该交易对中两种币可以转入，举例转入0.5个 BTC 与 1000 USDT

In [4]:
leverage_demo.deposit(0.5, 1000)

此时总资产即为转入资产

In [5]:
print(leverage_demo.total_asset_1, 'BTC', ',', leverage_demo.total_asset_2, 'USDT')

0.5 BTC , 1000 USDT


此时账户与现货交易账户完全相同，除了未成交挂单里冻结的资产外均可用于交易与转入转出，可转出资产为 **withdraw_available**

In [6]:
print(leverage_demo.withdraw_available_1, 'BTC', ',', leverage_demo.withdraw_available_2, 'USDT')

0.5 BTC , 1000 USDT


现在开始杠杆交易，通过类函数 **get_price()** 可得到此时 BTC/USDT 价格 **price** 为:

In [7]:
leverage_demo.get_price()
print(leverage_demo.price)

8127.549999999999


> get_price() 默认通过币安的 lastPrice 取得价格，实际应用时应通过交易所 order book 取得，且始终取对价，即卖出取买一价，买入取卖一价

客户存入的资金作为保证金，可从平台借贷资产获得更多可交易资产，加倍交易，可获得的最高倍数为杠杆倍数 **max_lever**

In [8]:
print(leverage_demo.max_lever, '倍')

3 倍


亦即存入一块钱作为保证金可以借到两块钱，一块钱可以当成三块钱用
客户存入的两种资产均可作为保证金，其价值由当前价格互相计算，如上存入 0.5 BTC 与 1000 USDT 的情况，按照当前价格，总资产则相当于：

以 BTC 计价：

In [9]:
leverage_demo.total_asset_1 + leverage_demo.total_asset_2 / leverage_demo.price

0.6230383079771887

或以 USDT 计价：

In [10]:
leverage_demo.total_asset_1 * leverage_demo.price + leverage_demo.total_asset_2

5063.775

据此可以计算 **calculate_cap** 可借的资产量 **borrow_available** ：

In [11]:
leverage_demo.calculte_cap()
print(leverage_demo.borrow_available_1, 'BTC', '或者', leverage_demo.borrow_available_2, 'USDT')

1.2460766159543775 BTC 或者 10127.55 USDT


如果此时看空 BTC，即认为 BTC/USDT 价会走低，则借 BTC 来卖出获得 USDT，等待价格走低再以 USDT 买回 BTC 归还，赚取差价
假设借 1 个BTC，利用 **loan** 函数，计入借贷记录 **all_loan** 并同时开始计算利息 **interest**

In [12]:
leverage_demo.loan(1, 0)

In [13]:
leverage_demo.all_loan

,time,loan_1,loan_2,interest_1,interest_2
1,1.523691e+09,1.0,0.0,0.001,0.0


日利率为 **interest_rate** ，即借出开始每 24 小时计算一次利息 **interest_grow**

In [14]:
print('借BTC利率为', leverage_demo.interest_rate_1, '，借USDT利率为', leverage_demo.interest_rate_2)

借BTC利率为 0.001 ，借USDT利率为 0.001


此时即可开始监控风险 **risk_monitor**

In [15]:
leverage_demo.risk_monitor()
leverage_demo.risk

Risk Rate            1.623032
STOP OUT PRICE   -2506.265664
Name: BTC/USDT, dtype: float64

In [16]:
leverage_demo.monitor

,Total,Borrowed,Borrow Available,Withdraw Available
btc,1.5,1,0.244065,-0.376968
usdt,1000,0,1983.74,-3063.97


In [17]:
print('风险率高于', leverage_demo.withdraw_risk_rate, '部分资产可取出，', \
      '低于', leverage_demo.margin_call_risk_rate, '将收到风险警告，', \
      '低于', leverage_demo.stop_out_risk_rate, '将被强制平仓')

风险率高于 2 部分资产可取出， 低于 1.5 将收到风险警告， 低于 1.1 将被强制平仓


实际上风险率为 1 即 100% 时即用户总资产等于借入资产（计入利息后），如此时不平仓，进一步亏损则客户可能无力归还借入资产，设为 110% 则有部分安全垫，此时有平台接管客户账户，可以通过减仓等操作使风险率符合规定或直接完全平仓，应根据市场所受冲击能力决定安全垫即强平风险阈值

客户借入资产后开始以转入资产和借入资产即总资产交易 **trade**，如上客户看空 BTC，则卖出，假设卖出 1.4 个 BTC

In [18]:
leverage_demo.trade(1.4,0)
leverage_demo.all_trade

,time,trade_1,trade_2,fee_1,fee_2
1,1.523691e+09,1.4,0.0,0.0014,0.0


之后客户持有资产与风险率等变化，通过 **risk_monitor** 监控：

In [19]:
leverage_demo.risk_monitor()
leverage_demo.risk

Risk Rate             1.621632
STOP OUT PRICE    12355.388489
Name: BTC/USDT, dtype: float64

In [20]:
leverage_demo.monitor

,Total,Borrowed,Borrow Available,Withdraw Available
btc,0.1,1,0.241265,-0.378368
usdt,12367.7,0,1960.99,-3075.35


可以看到，客户刚交易后总资产量除了手续费外没有变化，所以风险率也几乎没有变化，但由于持有的两种资产量变了，则预估爆仓价跟随变化，此时预估爆仓价为：

In [21]:
leverage_demo.stop_out_price

12355.38848851149

即 BTC/USDT 价格达到，风险率达到平仓风险率，账户有平台接管，强制减仓或平仓，如上借入 BTC 的情况，则由账户剩余资产买入 BTC 归还，如归还足额 BTC 后还有剩余，则返还到客户余额